In [64]:
%pip install spacy spacy-transformers
%pip install fuzzywuzzy
%pip install python-Levenshtein
%pip install transformers
%pip install gensim
%pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` befor

In [38]:

# !python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf



2023-09-06 12:36:54.476640: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 5.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [6]:
import spacy
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
from fuzzywuzzy import fuzz
from collections import defaultdict
import re

2023-09-06 13:28:51.130168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Functions

In [4]:
def preprocess_string(string : str) -> str:
    string = string.lower().strip()
    string = re.sub(' +', ' ', string)
    string = string.replace("_", " ")
    string = string.replace("-", " ")
    string = string.replace("&", " ")
    string = string.split("(")[0]

    # special cases
    if "computer" in string:
        string = "pc"

    if "tv" in string:
        string = "television"

    if "television" in string:
        string = "television"

    if "macbook" in string:
        string = "laptop"
        
    if "ev" == string:
        string = "car charger"
    
    if "toast" in string:
        string = "toaster"
    
    if "modem" in string:
        string = "router"

    # we treat all audio devices as speakers so subwoofer is also a speaker
    if "subwoofer" in string:
        string = "speaker"

    if "speaker" in string:
        string = "speaker"

    if "iron" in string:
        string = "iron"
    
    string = re.sub(' +', ' ', string)
    return string

### Get device list

In [7]:
device_list = []
path = "./Energy_graph/data/training_data/"
for file in tqdm(os.listdir(path)):
    if not file.endswith(".pkl"):
        continue
    
    print(file)
    data = pd.read_pickle(path + file)
    for house in data:
        
        for device in data[house]:
            device = preprocess_string(device)
            if device == "aggregate" or "outlet" in device or "sockets" in device or "lamp" in device or "light" in device or "plug" in device:
                continue
            test = device
            if "lamp" in device:
                print("HERE: ",house, device , test)
            if device not in device_list:
                device_list.append(device)


device_list = list(set(device_list))
    

  8%|▊         | 1/12 [00:00<00:01,  6.35it/s]

REDD.pkl
ECO.pkl


 17%|█▋        | 2/12 [00:04<00:24,  2.42s/it]

REFIT.pkl


 25%|██▌       | 3/12 [00:07<00:26,  2.94s/it]

...

 67%|██████▋   | 8/12 [00:16<00:06,  1.60s/it]

IAWE.pkl
HES.pkl


 83%|████████▎ | 10/12 [00:17<00:01,  1.02it/s]

DRED.pkl


100%|██████████| 12/12 [00:17<00:00,  1.47s/it]


### Fuzzy wuzzy

In [82]:

labels = device_list
grouped_labels = {}
threshold = 80 # Set a threshold for similarity

for label in labels:
    found = False
    for key in grouped_labels:
        if fuzz.ratio(label.lower(), key.lower()) > threshold:
            grouped_labels[key].append(label)
            found = True
            break
    if not found:
        grouped_labels[label] = [label]

# grouped_labels["fridge"].append("refrigerator")
# grouped_labels.pop("refrigerator")
grouped_labels, len(grouped_labels)

({'fridge': ['fridge'],
  'dish washer': ['dish washer', 'dishwasher'],
  'microwave': ['microwave'],
  'electric space heater': ['electric space heater',
   'electric heater',
   'electric heater '],
  'electric stove': ['electric stove', 'electric oven'],
  'washer dryer': ['washer dryer'],
  'waste disposal unit': ['waste disposal unit'],
  'ce appliance': ['ce appliance', 'wet appliance'],
  'electric furnace': ['electric furnace'],
  'smoke alarm': ['smoke alarm'],
  'air conditioner': ['air conditioner', 'air conditioning'],
  'subpanel': ['subpanel'],
  'air handling unit': ['air handling unit'],
  'pc': ['pc'],
  'washing machine': ['washing machine', 'washing machine '],
  'dryer': ['dryer'],
  'freezer': ['freezer', 'freezer '],
  'coffee machine': ['coffee machine', 'coffeemachine'],
  'kettle': ['kettle'],
  'laptop': ['laptop', 'laptops'],
  'router': ['router'],
  'air exhaust': ['air exhaust'],
  'stove': ['stove'],
  'stereo': ['stereo'],
  'tablet': ['tablet'],
  'tele

### Spacy using kmeans

In [62]:
import spacy
from sklearn.cluster import KMeans
import numpy as np

N_CLUSTERS = 54

# Load the largge English model in spaCy (contains word vectors)
# nlp = spacy.load('en_core_web_trf')
nlp = spacy.load('en_core_web_lg')

# List of devices
devices = device_list

# Create vectors for each device name
device_vectors = np.array([nlp(device).vector for device in devices])

# Use KMeans clustering to group devices
kmeans = KMeans(n_clusters=N_CLUSTERS)  # Choose an appropriate number of clusters
kmeans.fit(device_vectors)
labels = kmeans.labels_

# Print out the devices grouped by their cluster
grouped_devices = {}
for label, device in zip(labels, devices):
    if label not in grouped_devices:
        grouped_devices[label] = []
    grouped_devices[label].append(device)

for label, devices in grouped_devices.items():
    print(f"Group {label}: {devices}")


Group 7: ['fridge', 'freezer', 'fridge freezer', 'kimchi fridge', 'fridge freezer ', 'refrigerator', 'freezer ']
Group 36: ['broadband router', 'router']
Group 28: ['rice cooker', 'cooker']
Group 9: ['minioven', 'vaccumcleaner', 'subpanel', 'breadmaker', 'vivarium', 'coffeemachine', 'watercooker', 'handmixer', 'htpc']
Group 29: ['heat garage', 'heat basement', 'heat kitchen']
Group 45: ['wet appliance']
Group 49: ['upright freezer', 'chest freezer ', 'chest freezer']
Group 21: ['laptops', 'laptop', 'tablet']
Group 27: ['motor']
Group 50: ['waste disposal unit']
Group 4: ['heat bedroom #1', 'heat bedroom #2', 'heat bedroom #3']
Group 51: ['bouncy castle pump', 'pond pump']
Group 6: ['charger', 'mobile phone charger']
Group 25: ['washing machine', 'running machine', 'washing machine ']
Group 3: ['electric heating element', 'solar thermal pumping station', 'electric heater', 'electric heater ', 'heater', 'electric space heater', 'electric furnace', 'electric stove']
Group 38: ['tumble dry

### Spacy using kmeans using transformer

In [58]:
import spacy
from sklearn.cluster import KMeans
import numpy as np

# Initialize spaCy model with transformer
nlp = spacy.load("en_core_web_trf")

# Add the transformer component to the pipeline
if "transformer" not in nlp.pipe_names:
    transformer = nlp.add_pipe("transformer", name="transformer", last=True)

# Get vectors from transformer output
def get_vector_from_transformer_output(doc):
    tensor = doc._.trf_data.tensors[-1]
    return tensor.mean(axis=1).squeeze()

# Your list of devices
# device_list = ["laptop", "mobile", "tablet", "desktop"]

# Generate vectors
device_vectors = np.array([get_vector_from_transformer_output(nlp(device)) for device in device_list])


# KMeans clustering
# N_CLUSTERS = 2  # or however many clusters you'd like
kmeans = KMeans(n_clusters=N_CLUSTERS)
kmeans.fit(device_vectors.reshape(-1, 1))
labels = kmeans.labels_

# Group by labels
grouped_devices = {i: [] for i in set(labels)}
for label, device in zip(labels, device_list):
    grouped_devices[label].append(device)

# Display the groups
for label, devices in grouped_devices.items():
    print(f"Group {label}: {devices}")


Group 0: ['motor', 'air handling unit', 'smoke alarm', 'fridge freezer', 'blender', 'projector', 'car charger', 'air conditioning', 'set top box', 'fridge freezer ', 'bread maker']
Group 1: ['broadband router', 'dehumidifier/heater', 'gaming pc']
Group 2: ['solar thermal pumping station', 'tumble dryer ', 'tumble dryer']
Group 3: ['minioven', 'vivarium', 'heat kitchen']
Group 4: ['tablet', 'stereo', 'dish washer', 'handmixer']
Group 5: ['upright freezer', 'freezer', 'baby monitor', 'pc', 'electric furnace', 'freezer ', 'stove']
Group 6: ['fridge', 'waste disposal unit', 'charger', 'speaker', 'running machine', 'heat dining room', 'combination microwave', 'coffee machine', 'electric space heater', 'vacuum cleaner', 'pond pump', 'radio']
Group 7: ['wet appliance', 'microwave', 'audio amplifier', 'air conditioner', 'oven extractor fan', 'food mixer', 'watercooker']
Group 8: ['chest freezer ', 'washing machine', 'breadmaker', 'kimchi fridge', 'washing machine ', 'printer', 'coffeemachine',

### Genism k-means using google news word2vec

In [9]:
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
import numpy as np

N_CLUSTERS = 25
# Load pre-trained Word2Vec model
# Replace 'path/to/GoogleNews-vectors-negative300.bin' with the actual path to the pre-trained model
model = KeyedVectors.load_word2vec_format('./Energy_graph/google_word2vec/GoogleNews-vectors-negative300.bin', binary=True)

# List of words to cluster
words = device_list

# Create an array of word vectors
word_vectors = np.array([model[word] for word in words if word in model])

# Check for words not in the model
not_in_model = [word for word in words if word not in model]
if not_in_model:
    print(f"Note: The following words are not in the model: {not_in_model}")

# Perform KMeans clustering
 # You can choose a different number of clusters
kmeans = KMeans(n_clusters=N_CLUSTERS)
kmeans.fit(word_vectors)
labels = kmeans.labels_

# Print out the words grouped by their cluster
grouped_words = {}
for label, word in zip(labels, words):
    if label not in grouped_words:
        grouped_words[label] = []
    grouped_words[label].append(word)

for label, grouped_word in grouped_words.items():
    print(f"Group {label}: {grouped_word}")


Note: The following words are not in the model: ['coffeemachine', 'food mixer', 'electric oven', 'games console', 'electric stove', 'air conditioner', 'bouncy castle pump', 'electric heating element', 'food processor', 'hair straighteners', 'water purifier', 'audio system', 'tumble dryer', 'heat kitchen', 'heat bedroom #3', 'electric heater ', 'fridge freezer', 'kimchi fridge', 'heat basement', 'washing machine ', 'heat garage', 'whirlpool bath', 'office desk', 'combination microwave', 'electric heater', 'hi fi', 'solar thermal pumping station', 'htpc', 'air conditioning', 'usb hub', 'ce appliance', 'audio amplifier', 'air exhaust', 'handmixer', 'coffee maker', 'mobile phone charger', 'heat bedroom #2', 'tumble dryer ', 'set top box', 'stove oven', 'coffee machine', 'dish washer', 'air exchanger', 'baby monitor', 'running machine', 'hairdryer straightener', 'gaming pc', 'broadband router', 'chest freezer ', 'heat dining room', 'oven extractor fan', 'hair dryer', 'washing machine', 'pon